In [ ]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/MyDrive/Project/'
#%pwd

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1XSIbHWFizvrGifGPTwtsmagK2AS8ZE7n/Project


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [ ]:
np.random.seed(682)
tf.random.set_seed(682)

In [ ]:
class Distiller(keras.Model):
  
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results


In [ ]:
teacher = tf.keras.models.load_model('/saved_models/resnet50cifar')

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers import BatchNormalization
from keras.regularizers import l2

def alexnet_model(img_shape=(32, 32, 3), n_classes=10, l2_reg=0., weights=None):

	# Initialize model
	alexnet = Sequential()

	# Layer 1
	alexnet.add(Conv2D(96, (11, 11), input_shape=img_shape,
		padding='same', kernel_regularizer=l2(l2_reg)))
	alexnet.add(BatchNormalization())
	alexnet.add(Activation('relu'))
	alexnet.add(MaxPooling2D(pool_size=(2, 2)))

	# Layer 2
	alexnet.add(Conv2D(256, (5, 5), padding='same'))
	alexnet.add(BatchNormalization())
	alexnet.add(Activation('relu'))
	alexnet.add(MaxPooling2D(pool_size=(2, 2)))

	# Layer 3
	alexnet.add(ZeroPadding2D((1, 1)))
	alexnet.add(Conv2D(512, (3, 3), padding='same'))
	alexnet.add(BatchNormalization())
	alexnet.add(Activation('relu'))
	alexnet.add(MaxPooling2D(pool_size=(2, 2)))

	# Layer 4
	alexnet.add(ZeroPadding2D((1, 1)))
	alexnet.add(Conv2D(1024, (3, 3), padding='same'))
	alexnet.add(BatchNormalization())
	alexnet.add(Activation('relu'))

	# Layer 5
	alexnet.add(ZeroPadding2D((1, 1)))
	alexnet.add(Conv2D(1024, (3, 3), padding='same'))
	alexnet.add(BatchNormalization())
	alexnet.add(Activation('relu'))
	alexnet.add(MaxPooling2D(pool_size=(2, 2)))

	# Layer 6
	alexnet.add(Flatten())
	alexnet.add(Dense(3072))
	alexnet.add(BatchNormalization())
	alexnet.add(Activation('relu'))
	alexnet.add(Dropout(0.5))

	# Layer 7
	alexnet.add(Dense(4096))
	alexnet.add(BatchNormalization())
	alexnet.add(Activation('relu'))
	alexnet.add(Dropout(0.5))

	# Layer 8
	alexnet.add(Dense(n_classes))

	if weights is not None:
		alexnet.load_weights(weights)

	return alexnet

In [ ]:
# Create the student
student = alexnet_model()

# Clone student for later comparison
student_scratch = keras.models.clone_model(student)

In [ ]:
# Prepare the train and test dataset.
batch_size = 64

(x_train, y_train) , (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

170508288/170498071 [==============================] - 11s 0us/step


In [ ]:
# Initialize and compile distiller
distiller = Distiller(student=student, teacher=teacher)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)

# Distill teacher to student
distiller.fit(x_train, y_train, epochs=5)

# Evaluate student on test dataset
distiller.evaluate(x_test, y_test)

Epoch 1/5
1563/1563 [==============================] - 150s 88ms/step - sparse_categorical_accuracy: 0.3460 - student_loss: 2.1014 - distillation_loss: 0.0177
Epoch 2/5
1563/1563 [==============================] - 137s 87ms/step - sparse_categorical_accuracy: 0.5085 - student_loss: 1.6514 - distillation_loss: 0.0203
Epoch 3/5
1563/1563 [==============================] - 136s 87ms/step - sparse_categorical_accuracy: 0.5997 - student_loss: 1.3513 - distillation_loss: 0.0185
Epoch 4/5
1563/1563 [==============================] - 136s 87ms/step - sparse_categorical_accuracy: 0.6621 - student_loss: 1.0907 - distillation_loss: 0.0159
Epoch 5/5
313/313 [==============================] - 5s 15ms/step - sparse_categorical_accuracy: 0.6152 - student_loss: 1.4366


[0.6151999831199646, 0.7269378304481506]

In [ ]:
student.save('/saved_models/ts_1')

student_test = tf.keras.models.load_model('/saved_models/ts_1')

# Initialize and compile distiller
distiller_test = Distiller(student=student_test, teacher=teacher)
metric_student = keras.metrics.SparseCategoricalAccuracy(name='s1_acc')

distiller_test.compile (
    optimizer=keras.optimizers.Adam(),
    metrics=[metric_student],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)

# Evaluate student on test dataset
distiller_test.evaluate(x_test, y_test)

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Project/models/saved_models/ts_1/assets
313/313 [==============================] - 5s 15ms/step - s1_acc: 0.6152 - student_loss: 1.4366


[0.6151999831199646, 0.7269378304481506]

In [ ]:
# Train student as doen usually
student_scratch.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train and evaluate student trained from scratch.
student_scratch.fit(x_train, y_train, epochs=5)
student_scratch.evaluate(x_test, y_test)

Epoch 1/5
1563/1563 [==============================] - 61s 38ms/step - loss: 2.1401 - sparse_categorical_accuracy: 0.3569
Epoch 2/5
1563/1563 [==============================] - 59s 38ms/step - loss: 1.6944 - sparse_categorical_accuracy: 0.4990
Epoch 3/5
1563/1563 [==============================] - 59s 38ms/step - loss: 1.3992 - sparse_categorical_accuracy: 0.5735
Epoch 4/5
1563/1563 [==============================] - 59s 38ms/step - loss: 1.1231 - sparse_categorical_accuracy: 0.6431
Epoch 5/5
313/313 [==============================] - 4s 13ms/step - loss: 1.4385 - sparse_categorical_accuracy: 0.6054


[1.4384832382202148, 0.605400025844574]

In [ ]:
student_scratch.save('/saved_models/scratch_1')

student_scratch_test = tf.keras.models.load_model('/saved_models/scratch_1')

# Train student as doen usually
student_scratch_test.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train and evaluate student trained from scratch.
student_scratch_test.evaluate(x_test, y_test)

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Project/models/saved_models/scratch_1/assets
313/313 [==============================] - 5s 14ms/step - loss: 1.4385 - sparse_categorical_accuracy: 0.6054


[1.4384832382202148, 0.605400025844574]

In [ ]:
# Initialize and compile distiller
distiller = Distiller(student=student, teacher=teacher)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)

# Distill teacher to student
distiller.fit(x_train, y_train, epochs=10)

# Evaluate student on test dataset
distiller.evaluate(x_test, y_test)

Epoch 1/10
1563/1563 [==============================] - 138s 87ms/step - sparse_categorical_accuracy: 0.3933 - student_loss: 1.9667 - distillation_loss: 0.0188
Epoch 2/10
1563/1563 [==============================] - 135s 87ms/step - sparse_categorical_accuracy: 0.5262 - student_loss: 1.5642 - distillation_loss: 0.0189
Epoch 3/10
1563/1563 [==============================] - 135s 87ms/step - sparse_categorical_accuracy: 0.6062 - student_loss: 1.2900 - distillation_loss: 0.0163
Epoch 4/10
1563/1563 [==============================] - 136s 87ms/step - sparse_categorical_accuracy: 0.6764 - student_loss: 1.0227 - distillation_loss: 0.0138
Epoch 5/10
1563/1563 [==============================] - 136s 87ms/step - sparse_categorical_accuracy: 0.7285 - student_loss: 0.8531 - distillation_loss: 0.0127
Epoch 6/10
1563/1563 [==============================] - 135s 87ms/step - sparse_categorical_accuracy: 0.7683 - student_loss: 0.7288 - distillation_loss: 0.0121
Epoch 7/10
1563/1563 [==================

[0.7282000184059143, 0.8351705074310303]

In [ ]:
# Train student as doen usually
student_scratch.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train and evaluate student trained from scratch.
student_scratch.fit(x_train, y_train, epochs=10)
student_scratch.evaluate(x_test, y_test)

Epoch 1/10
1563/1563 [==============================] - 58s 37ms/step - loss: 2.2172 - sparse_categorical_accuracy: 0.3405
Epoch 2/10
1563/1563 [==============================] - 57s 37ms/step - loss: 1.8121 - sparse_categorical_accuracy: 0.4735
Epoch 3/10
1563/1563 [==============================] - 57s 37ms/step - loss: 1.5284 - sparse_categorical_accuracy: 0.5545
Epoch 4/10
1563/1563 [==============================] - 57s 36ms/step - loss: 1.2795 - sparse_categorical_accuracy: 0.6164
Epoch 5/10
1563/1563 [==============================] - 57s 36ms/step - loss: 1.0264 - sparse_categorical_accuracy: 0.6691
Epoch 6/10
1563/1563 [==============================] - 57s 36ms/step - loss: 0.8437 - sparse_categorical_accuracy: 0.7203
Epoch 7/10
1563/1563 [==============================] - 57s 37ms/step - loss: 0.7214 - sparse_categorical_accuracy: 0.7604
Epoch 8/10
1563/1563 [==============================] - 57s 37ms/step - loss: 0.6038 - sparse_categorical_accuracy: 0.7968
Epoch 9/10
1563/

[0.8228883743286133, 0.7369999885559082]